In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
crash_data = pd.read_csv("crash_data.csv", usecols=(0,1,2,3,4,5))    #Reading data
crash_data.head()

,country,date,fatalities,flight_model,location,operator
0,Italy,02-08-1919,14,Caproni Ca.48,Verona,Caproni
1,U.K.,11-08-1919,1,Felixstowe Fury,near Felixtowe RNAS,RAF
2,South Africa,23-02-1920,0,Handley Page O/7,Acadia Sidin...,Handley Page Transport
3,Sudan,25-02-1920,0,Handley Page O/400,near El Shereik,Handley Page Transport
4,Sweden,30-06-1920,0,Handley Page O/400,CstanC%,Handley Page Transport


In [6]:
crash_data.isnull().sum()

country            0
date               1
fatalities      4163
flight_model       2
location         964
operator          60
dtype: int64

In [9]:
crash_data["date"].head(40)

0      02-08-1919
1      11-08-1919
2      23-02-1920
3      25-02-1920
4      30-06-1920
5      14-12-1920
6      15-01-1921
7      02-03-1921
8      04-03-1921
9      26-08-1921
10     27-09-1921
11     27-09-1921
12     30-12-1921
13     14-01-1922
14     31-03-1922
15     07-04-1922
16     13-01-1923
17     14-05-1923
18     25-06-1923
19     10-07-1923
20     15-07-1923
21     27-08-1923
22     03-12-1923
23    ??-???-1923
24     22-01-1924
25     06-08-1924
26     24-08-1924
27     16-10-1925
28     14-11-1925
29     18-08-1926
30     02-10-1926
31     21-10-1926
32     20-11-1926
33     05-05-1927
34     22-08-1927
35     15-02-1928
36     11-03-1928
37     11-04-1928
38     19-04-1928
39     12-05-1928
Name: date, dtype: object

In [3]:
crash_data["fatalities"].nunique()

364

In [4]:
crash_data["fatalities"].value_counts()

0         9307
2         1024
1          913
3          893
4          770
          ... 
114          1
29+ 1        1
0+ 12        1
7+ 4         1
109+ 4       1
Name: fatalities, Length: 364, dtype: int64

In [5]:
for i in range(0,len(crash_data["fatalities"])):
        value1=(crash_data["fatalities"][i].str.split("+").str[0]).astype(int32)
        value2=(crash_data["fatalities"][i].str.split("+").str[1]).astype(int32)
        if (value2 != null):
            crash_data["fatalities"][i] = (value1[i]+value2[i])/2


AttributeError: 'str' object has no attribute 'str'

In [38]:
value2.notna().sum()

335

country            0
date               1
fatalities      4163
flight_model       2
location         964
operator          60
dtype: int64